In [ ]:
from pyannote.audio import Pipeline
import pickle
import os
import time

In [ ]:
#assuming the first person speaking is always the interviewer and there are only 2 people speaking
def getInteviewerAndProbantSegments(diarization):
    interviewer = ""
    interviewer_segments = []
    probant = ""
    probant_segments = []
    for i, segment in enumerate(diarization.itersegments()):
        if i == 0:
            interviewer = diarization.get_labels(segment)
        if interviewer == diarization.get_labels(segment):
            interviewer_segments.append(segment)   
        else:
            probant_segments.append(segment)
    return (interviewer_segments, probant_segments)

In [ ]:
def safeDiarization(diarization, interviewer_segments, probant_segments, probant_acronym):
    
    filename_diarization = f"{probant_acronym}_diarization.pkl"
    filename_diarization_rttm = f"{probant_acronym}_diarization.rttm"
    filepath = os.path.join('../data/annotation_objects/', filename_diarization)
    with open(filepath, 'wb') as file:
        pickle.dump(diarization, file)
    filepath = os.path.join('../data/annotation_objects/', filename_diarization_rttm)
    with open(filepath, "w") as rttm:
        diarization.write_rttm(rttm)
        
    filename_interviewer_segments = f"{probant_acronym}_interviewer_segments.pkl"
    filepath = os.path.join('../data/annotation_objects/', filename_interviewer_segments)
    with open(filepath, 'wb') as file:
        pickle.dump(interviewer_segments, file)
        
    filename_probant_segments = f"{probant_acronym}_probant_segments.pkl"
    filepath = os.path.join('../data/annotation_objects/', filename_probant_segments)
    with open(filepath, 'wb') as file:
        pickle.dump(probant_segments, file)

In [ ]:
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization@2.1",
                                    use_auth_token="...")

In [ ]:
parent_folder = '../data/20221121_Sprachaufnahmen_Preprocessed'

for root, dirs, files in os.walk(parent_folder):
    for folder_name in dirs:
        if 'Interview' in folder_name:
            subfolder_path = os.path.join(root, folder_name)
            person = folder_name.split("_", 1)[0]
            person_time_point = folder_name.split("_", -1)[-1]
            probant_acronym = person + "_" + person_time_point
            print(folder_name)
            for file_name in os.listdir(subfolder_path):
                print(file_name)
                if 'ZOO' in file_name and not file_name.endswith('BU.WAV') and (file_name.endswith('.WAV') or file_name.endswith('.wav')):
                    
                    annotation_folder = '../data/annotation_objects'
                    annotation_file = os.path.join(annotation_folder, f'{probant_acronym}_diarization.rttm')

                    if os.path.exists(annotation_file):
                        print(f'Skipping pipeline for {probant_acronym} (annotation file already exists)\n')
                        continue
                                        
                    print(f'Run pipeline for {probant_acronym}')
                    
                    print(f'Started diarizing')
                    start_time = time.time()
                    file_path = os.path.join(subfolder_path, file_name)
                    diarization = pipeline(file_path, num_speakers=2)
                    end_time = time.time()
                    print(f'Finished diarizing after {end_time - start_time} s')
                    
                    interviewer_segments, probant_segments = getInteviewerAndProbantSegments(diarization)
                    
                    safeDiarization(diarization, interviewer_segments, probant_segments, probant_acronym)
                    print(f'Diarization safed')
                    
                    print(f'Finished pipeline run for {probant_acronym}\n')